## De Opdracht - Mesa
Er is een enorme hoeveelheid tools ontworpen waarmee je god kan spelen over je eigen werkelijkheid. Zie dit paper voor meer informatie over deze tools. Deze week maken jullie kennis met een van deze tools. De opdracht is individueel, maar voor het team is het uitermate belangrijk dat je deze opdracht serieus aanpakt! Volgende week gaan jullie namelijk werken aan een plan van aanpak voor een onderzoek naar verkeerssituaties of verkiezingstrategien met behulp van een Agent-based simulatie. Daarbij moeten jullie aan elkaar kunnen uitleggen wat de voor- en nadelen zijn van de tool die je deze week gaat ontdekken.

De opdracht bestaat uit het volgen van een tutorial voor één van drie tools om een Agent-based simulatie te maken. Bij de groepsopdracht wordt verwacht dat alle drie de tools meegenomen worden bij een beslissing over welke tool gebruikt gaat worden voor de opdracht. Dus het helpt als je als team gezamelijk kennis heeft over alle tools.

### Mesa Overview (Tutorial)
Deze tutorial is te vinden met de link:

https://mesa.readthedocs.io/en/master/overview.html

We beginnen eerst met de imports nodig om dit te laten werken.

In [1]:
# Imports
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid

from mesa.datacollection import DataCollector
from mesa.batchrunner import BatchRunner

from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer

Dan gaan we het skelet van een Mesa model opzetten.
We beginnen met een basic agent.

In [2]:
class MyAgent(Agent):
    def __init__(self, name, model):
        super().__init__(name, model)
        self.name = name
        # Dit maakt het agent object aan.

    def step(self):
        print("{} activated".format(self.name))
        # Step is de functie die aangeeft wat de agent doet als deze wordt aangeroepen.

De agent doet momenteel nog niks buiten het aangeven dat hij wordt aangeroepen.

Dan maken we een basic model waarin de agents kunnen bestaan. Dit is echter nog geen grid waar de agents op kunnen staan. Het is een soort main functie waar alles zich in bevindt.

In [3]:
class MyModel(Model):
    def __init__(self, n_agents):
        super().__init__()
        self.schedule = RandomActivation(self) # Hier is de schedule RandomActivation aangemaakt die en step() wordt benoemd.
        self.grid = MultiGrid(10, 10, torus=True) # Een grid waar agents op geplaatst kunnen worden.
        for i in range(n_agents):
            a = MyAgent(i, self)
            self.schedule.add(a)
            coords = (self.random.randrange(0, 10), self.random.randrange(0, 10))
            self.grid.place_agent(a, coords)

    def step(self):
        self.schedule.step()
         # RandomActivation activeerd elke stap een random agent. En is te vinden in de time module

Het bovenstaande model maakt dus voor het aantal aangegeven agents agents aan in een loop en plaats deze ergens random op de grid. Als de step functie wordt aangeroepen komt het model pas 'tot leven'. Tot die tijd doen de agents dus nog niets.

Hieronder wordt het model aangemaakt en 1 step geactiveerd.

In [4]:
model = MyModel(5)
model.step()

3 activated
0 activated
1 activated
2 activated
4 activated


Ook kan je een data collector toevoegen.

In [5]:
class MyDataCollectorModel(Model):
    def __init__(self, n_agents):
        # ...
        self.dc = DataCollector(model_reporters={"agent_count":
                                    lambda m: m.schedule.get_agent_count()},
                                agent_reporters={"name": lambda a: a.name})

    def step(self):
        self.schedule.step()
        self.dc.collect(self) # Hier zie je dat de data dus elke step wordt opgeslagen.

Dit voegen we toen aan ons bestaande model:

In [6]:
class MyModel(Model):
    def __init__(self, n_agents):
        super().__init__()
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(10, 10, torus=True)
        self.dc = DataCollector(model_reporters={"agent_count": # <<<
                                    lambda m: m.schedule.get_agent_count()},
                                agent_reporters={"name": lambda a: a.name})
        for i in range(n_agents):
            a = MyAgent(i, self)
            self.schedule.add(a)
            coords = (self.random.randrange(0, 10), self.random.randrange(0, 10))
            self.grid.place_agent(a, coords)

    def step(self):
        self.schedule.step()
        self.dc.collect(self) # <<<

In [7]:
model = MyModel(5)
for t in range(2):
    model.step()
model_df = model.dc.get_model_vars_dataframe()
agent_df = model.dc.get_agent_vars_dataframe()

1 activated
0 activated
2 activated
3 activated
4 activated
2 activated
3 activated
4 activated
0 activated
1 activated


Hier onder kan je zien wat het resultaat is van de data collectors.

In [8]:
model_df

,agent_count
0,5
1,5


In [9]:
agent_df

name
Step AgentID      
1    0           0
     1           1
     2           2
     3           3
     4           4
2    0           0
     1           1
     2           2
     3           3
     4           4

dan heb je ook nog een batch runner wat er voor zorgt dat je een model meerdere keren achter elkaar kan laten runnen met andere parameters. 

(In dit geval dus steeds meer agents.)

In [10]:
parameters = {"n_agents": range(1, 20)}
batch_run = BatchRunner(MyModel, parameters, max_steps=10,
                        model_reporters={"n_agents": lambda m: m.schedule.get_agent_count()})

In [11]:
# Run alle batches.
batch_run.run_all()

0it [00:00, ?it/s]

0 activated
0 activated
0 activated
0 activated
0 activated
0 activated
0 activated
0 activated
0 activated
0 activated
0 activated
1 activated
0 activated
1 activated
0 activated
1 activated
0 activated
1 activated
1 activated
0 activated
1 activated
0 activated
0 activated
1 activated
1 activated
0 activated
1 activated
0 activated
1 activated
0 activated
2 activated
0 activated
1 activated
1 activated
2 activated
0 activated
1 activated
2 activated
0 activated
0 activated
1 activated
2 activated
2 activated
0 activated
1 activated
2 activated
1 activated
0 activated
2 activated
1 activated
0 activated
1 activated
0 activated
2 activated
0 activated
2 activated
1 activated
0 activated
2 activated
1 activated
0 activated
2 activated
1 activated
3 activated
0 activated
3 activated
2 activated
1 activated
3 activated
1 activated
2 activated
0 activated
1 activated
2 activated
0 activated
3 activated
1 activated
3 activated
2 activated
0 activated
0 activated
1 activated
3 activated
2 ac

19it [00:00, 216.48it/s]


7 activated
15 activated
0 activated
11 activated
4 activated
13 activated
16 activated
1 activated
6 activated
9 activated
14 activated
12 activated
3 activated
2 activated
5 activated
10 activated
8 activated
15 activated
13 activated
14 activated
6 activated
3 activated
16 activated
12 activated
5 activated
2 activated
9 activated
7 activated
8 activated
0 activated
10 activated
4 activated
11 activated
1 activated
1 activated
6 activated
5 activated
16 activated
13 activated
3 activated
2 activated
11 activated
12 activated
9 activated
15 activated
0 activated
4 activated
10 activated
7 activated
14 activated
8 activated
14 activated
12 activated
13 activated
7 activated
9 activated
0 activated
3 activated
5 activated
6 activated
1 activated
16 activated
10 activated
15 activated
11 activated
8 activated
4 activated
2 activated
16 activated
2 activated
10 activated
14 activated
11 activated
7 activated
5 activated
6 activated
12 activated
3 activated
4 activated
13 activated
15 ac

Ook van een batchrun is een data collector te maken. (Hier zie je dat het aantal agents per run daadwerkelijk meer wordt.)

In [12]:
batch_df = batch_run.get_model_vars_dataframe()
batch_df

,n_agents,Run
0,1,0
1,2,1
2,3,2
3,4,3
4,5,4
5,6,5
6,7,6
7,8,7
8,9,8
9,10,9


En dan tot slot het visualiseren van het model.
Om een weergave van de agent te maken kan je iets simpels als hier onder maken.

In [13]:
def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "Layer": 0,
                 "Color": "red",
                 "r": 0.5}
    return portrayal

Waarna je een grid kan maken waarin dus de agents met bovenstaande functie worden gevisualiseerd.

De grid is nu nog niet zelf zichtbaar als je hem aanroept.

In [14]:
grid = CanvasGrid(agent_portrayal, 10, 10, 500, 500)
grid

Als je wilt dat het zichtbaar wordt moet je eerst een server aanmaken en deze vervolgens lauchen.

In [16]:
server = ModularServer(MyModel,
                       [grid],
                       "My Model",
                       {'n_agents': 10})
# server.launch()

Helaas is er geen stop funcite voor de server en kan je deze niet meerdere keren achter elkaar aanroepen.
Als je dat wel wilt doen is het nodig om de kernel te herstarten.

Tot hier gaat de Mesa overview tutorial.

## Vragen
1. Volg de tutorial en omschrijf daarna in één paragraaf wat deze tool anders maakt dan andere programmeertalen, wat zijn de voor- en nadelen? Is het om een of andere reden niet gelukt de tutorial af te maken: beschrijf waar het mis ging. Als je een bestaand voorbeeld gebruikt (Netlogo heeft er vele): beschrijf de kernfunctionaliteit van het voorbeeld en voeg minimaal een element toe aan de omgeving en licht toe. (bijvoorbeeld: het meest eenvoudige economiemodel in Netlogo is agenten die random geld uitdelen, je kunt dat aanpassen naar geld uitdelen aan agenten die dichtbij hen staan.

In de tutorial van Mesa viel het meteen op dat de gebruiksvriendelijkheid erg hoog is. Mesa is een python module en dus erg makkelijk te leren voor mensen met kennis van Python. Ook is Mesa een erg goede keuze voor het verzamelen van simulatie data aangezien je in 1-2 lines code een compleet gevulde pandas dataframe hebt. In mijn ogen ligt hier ook de kracht van Mesa aangezien dit ook te combineren is met de batchrunner. Een nadeel aan mesa is dat de weergave persee via een server moet die niet heel aanpasbaar is. Je kan de agents en grid wel aanpassen of 